# Emergency Room Efficiency Analysis## Meridian City Hospital - East ER Location**Objective:** Identify bottlenecks causing ER delays and provide actionable recommendations**Key Metrics:**- Target: 40%+ of patients seen within 15 minutes- Current average wait time: 45+ minutes- Current average total ER time: 2.5 hours---

## 1. Setup & ImportsImporting all required libraries and creating outputs folder.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score,
                              classification_report, confusion_matrix)

# Settings
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
warnings.filterwarnings('ignore')

# Create outputs directory
os.makedirs('outputs', exist_ok=True)
print("✅ Libraries imported successfully")
print("✅ Outputs folder ready")

## 2. Data LoadingLoading all CSV datasets for analysis.

In [ ]:
# Load datasets
patients = pd.read_csv('data/Hospital_Patients.csv')
visits = pd.read_csv('data/Hospital_Visits.csv')
staffing = pd.read_csv('data/Hospital_Staffing_EAST_LOCATION.csv')
facility = pd.read_csv('data/Hospital_Facility.csv')
outcomes = pd.read_csv('data/Hospital_Outcomes.csv')

print(f"📊 Patients: {len(patients):,} records")
print(f"📊 Visits: {len(visits):,} records")
print(f"📊 Staffing: {len(staffing):,} records")
print(f"📊 Outcomes: {len(outcomes):,} records")
print(f"📊 Facility: {len(facility):,} records")

In [ ]:
# Display sample data
print("\n" + "="*60)
print("SAMPLE DATA")
print("="*60)
display(visits.head(3))

## 3. Data CleaningHandling inconsistent formats:- Mixed date formats (3/9/2025 3:44, Mar 09 2025 03:21, 2025-03-09T03:27)- Inconsistent triage levels (moderate, 2, 1, low, urgent)- Inconsistent dispositions (Discharged, Disch, ADM, transfer)- Mixed shift capitalization (Day, NIGHT, evening)

In [ ]:
def parse_flexible_date(date_str):
    """Parse multiple date formats"""
    if pd.isna(date_str):
        return pd.NaT
    
    formats = [
        '%m/%d/%Y %H:%M', '%b %d %Y %H:%M', '%Y-%m-%dT%H:%M',
        '%Y-%m-%d %H:%M:%S', '%m/%d/%Y %H:%M:%S'
    ]
    
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
    
    try:
        return pd.to_datetime(date_str)
    except:
        return pd.NaT

print("✅ Date parser function defined")

In [ ]:
# Filter for East ER only
print(f"Total visits before filtering: {len(visits):,}")
visits = visits[visits['Hospital ID'] == 'MC_ER_EAST'].copy()
print(f"East ER visits: {len(visits):,}")

# Parse timestamps
timestamp_cols = ['Arrival Time', 'Registration Start', 'Registration End',
                  'Triage Start', 'Triage End', 'Doctor Seen', 'Exit Time']

print("\n🔧 Parsing timestamps...")
for col in timestamp_cols:
    visits[col] = visits[col].apply(parse_flexible_date)

print("✅ Timestamps parsed")

In [ ]:
# Standardize triage levels
def standardize_triage(level):
    if pd.isna(level):
        return 'Unknown'
    level = str(level).lower().strip()
    if level == '1' or 'immediate' in level or 'critical' in level:
        return 'Level 1 - Immediate'
    elif level == '2' or 'emergency' in level or 'emergent' in level:
        return 'Level 2 - Emergency'
    elif level == '3' or 'urgent' in level:
        return 'Level 3 - Urgent'
    elif level == '4' or 'semi' in level or 'moderate' in level:
        return 'Level 4 - Semi-urgent'
    elif 'low' in level or 'nonurgent' in level:
        return 'Level 5 - Non-urgent'
    return 'Unknown'

visits['Triage Level'] = visits['Triage Level'].apply(standardize_triage)
print("Triage distribution:")
print(visits['Triage Level'].value_counts())

In [ ]:
# Standardize shifts and dispositions
staffing['Shift'] = staffing['Shift'].str.capitalize()
staffing['Date'] = pd.to_datetime(staffing['Date'])

# Standardize disposition
def standardize_disposition(disp):
    if pd.isna(disp):
        return 'Unknown'
    disp = str(disp).lower().strip()
    if 'disch' in disp:
        return 'Discharged'
    elif 'adm' in disp or 'admit' in disp:
        return 'Admitted'
    elif 'transfer' in disp:
        return 'Transferred'
    return 'Unknown'

outcomes['Disposition'] = outcomes['Disposition'].apply(standardize_disposition)
print("\n✅ All data cleaned successfully")

## 4. Feature EngineeringCreating calculated fields for analysis.

In [ ]:
# Calculate wait times (in minutes)
visits['Wait_to_Registration'] = (visits['Registration Start'] - visits['Arrival Time']).dt.total_seconds() / 60
visits['Registration_Duration'] = (visits['Registration End'] - visits['Registration Start']).dt.total_seconds() / 60
visits['Wait_to_Triage'] = (visits['Triage Start'] - visits['Registration End']).dt.total_seconds() / 60
visits['Triage_Duration'] = (visits['Triage End'] - visits['Triage Start']).dt.total_seconds() / 60
visits['Wait_to_Doctor'] = (visits['Doctor Seen'] - visits['Triage End']).dt.total_seconds() / 60
visits['Treatment_Duration'] = (visits['Exit Time'] - visits['Doctor Seen']).dt.total_seconds() / 60
visits['Total_ER_Time'] = (visits['Exit Time'] - visits['Arrival Time']).dt.total_seconds() / 60
visits['Time_to_Doctor'] = (visits['Doctor Seen'] - visits['Arrival Time']).dt.total_seconds() / 60

# 15-minute target flag
visits['Seen_Within_15min'] = (visits['Time_to_Doctor'] <= 15).astype(int)

print("✅ Wait times calculated")
print(f"Avg time to doctor: {visits['Time_to_Doctor'].mean():.1f} min")
print(f"% within 15 min: {visits['Seen_Within_15min'].mean()*100:.1f}%")

In [ ]:
# Temporal features
visits['Date'] = visits['Arrival Time'].dt.date
visits['Hour'] = visits['Arrival Time'].dt.hour
visits['Day_of_Week'] = visits['Arrival Time'].dt.day_name()
visits['Is_Weekend'] = visits['Arrival Time'].dt.dayofweek.isin([5, 6]).astype(int)

def assign_shift(hour):
    if 7 <= hour < 15:
        return 'Day'
    elif 15 <= hour < 23:
        return 'Evening'
    else:
        return 'Night'

visits['Shift'] = visits['Hour'].apply(assign_shift)
print("✅ Temporal features created")

In [ ]:
# Merge datasets
visits = visits.merge(patients, on='Patient ID', how='left')
visits = visits.merge(outcomes, on='Visit ID', how='left')

visits['Date_dt'] = pd.to_datetime(visits['Date'])
staffing['Date'] = pd.to_datetime(staffing['Date'])

visits = visits.merge(staffing, left_on=['Date_dt', 'Shift'],
                      right_on=['Date', 'Shift'], how='left', suffixes=('', '_staff'))

# Calculate staffing ratios
visit_counts = visits.groupby(['Date_dt', 'Shift']).size().reset_index(name='Patient_Count')
staffing_merged = staffing.merge(visit_counts, left_on=['Date', 'Shift'],
                                  right_on=['Date_dt', 'Shift'], how='left')

staffing_merged['Patient_Count'] = staffing_merged['Patient_Count'].fillna(0)
staffing_merged['Patients_per_Nurse'] = staffing_merged['Patient_Count'] / staffing_merged['Nurses On Duty']
staffing_merged['Patients_per_Doctor'] = staffing_merged['Patient_Count'] / staffing_merged['Doctors On Duty']

visits = visits.merge(staffing_merged[['Date', 'Shift', 'Patients_per_Nurse', 'Patients_per_Doctor']],
                      left_on=['Date_dt', 'Shift'], right_on=['Date', 'Shift'],
                      how='left', suffixes=('', '_ratio'))

print(f"✅ Datasets merged: {visits.shape}")

## 5. Exploratory Data Analysis

In [ ]:
# Key metrics
print("="*70)
print("KEY PERFORMANCE METRICS")
print("="*70)
print(f"Average time to doctor: {visits['Time_to_Doctor'].mean():.1f} min")
print(f"% seen within 15 min: {visits['Seen_Within_15min'].mean()*100:.1f}%")
print(f"Average total ER time: {visits['Total_ER_Time'].mean()/60:.1f} hours")
print(f"Average satisfaction: {visits['Patient Satisfaction'].mean():.2f}/5.0")

In [ ]:
# Wait Time Breakdown Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Wait time components
components = [visits['Wait_to_Registration'].mean(), visits['Registration_Duration'].mean(),
              visits['Wait_to_Triage'].mean(), visits['Triage_Duration'].mean(),
              visits['Wait_to_Doctor'].mean(), visits['Treatment_Duration'].mean()]

labels = ['Wait to\nReg', 'Registration', 'Wait to\nTriage', 'Triage', 'Wait to\nDoctor', 'Treatment']
colors = ['#ff6b6b' if 'Wait' in l else '#4ecdc4' for l in labels]

axes[0,0].bar(labels, components, color=colors, edgecolor='black')
axes[0,0].set_ylabel('Minutes')
axes[0,0].set_title('Average Time by ER Stage', fontweight='bold')

# 2. Time-to-doctor distribution
axes[0,1].hist(visits['Time_to_Doctor'], bins=50, edgecolor='black', alpha=0.7)
axes[0,1].axvline(15, color='red', linestyle='--', linewidth=2, label='15-min target')
axes[0,1].set_xlabel('Minutes')
axes[0,1].set_ylabel('Count')
axes[0,1].set_title('Time to See Doctor Distribution', fontweight='bold')
axes[0,1].legend()

# 3. Performance by triage
triage_perf = visits.groupby('Triage Level')['Seen_Within_15min'].mean() * 100
axes[1,0].barh(triage_perf.index, triage_perf.values, color='coral', edgecolor='black')
axes[1,0].set_xlabel('% Seen Within 15 Min')
axes[1,0].set_title('Performance by Triage Level', fontweight='bold')

# 4. Arrivals by hour
hourly = visits.groupby('Hour').size()
axes[1,1].plot(hourly.index, hourly.values, marker='o', linewidth=2, markersize=8)
axes[1,1].set_xlabel('Hour of Day')
axes[1,1].set_ylabel('Arrivals')
axes[1,1].set_title('Patient Arrivals by Hour', fontweight='bold')
axes[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/wait_time_analysis.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Saved: outputs/wait_time_analysis.png")

## 6. Statistical Analysis & Correlations

In [ ]:
# Correlation matrix
corr_features = ['Time_to_Doctor', 'Total_ER_Time', 'Patient Satisfaction',
                 'Nurses On Duty', 'Doctors On Duty', 'Patients_per_Nurse',
                 'Patients_per_Doctor', 'Age', 'Hour']

corr_data = visits[corr_features].dropna()
corr_matrix = corr_data.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1)
plt.title('Correlation Matrix: Key Variables', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Saved: outputs/correlation_matrix.png")

## 7. Machine Learning Models### 7.1 Model #1: Wait Time Prediction (Regression)

In [ ]:
# Prepare data for ML
feature_cols = ['Hour', 'Is_Weekend', 'Nurses On Duty', 'Doctors On Duty',
                'Fast Track Beds', 'Age', 'Patients_per_Nurse', 'Patients_per_Doctor']

ml_data = visits[feature_cols + ['Time_to_Doctor', 'Total_ER_Time', 'Patient Satisfaction']].dropna()

X = ml_data[feature_cols]
y = ml_data['Time_to_Doctor']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train models
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Linear Regression': LinearRegression()
}

results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    results[name] = {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'model': model}
    print(f"{name}: MAE={mae:.2f}, RMSE={rmse:.2f}, R²={r2:.3f}")

best_model_name = min(results, key=lambda x: results[x]['MAE'])
best_model = results[best_model_name]['model']
print(f"\n✅ Best Model: {best_model_name} (MAE: ±{results[best_model_name]['MAE']:.1f} min)")

### 7.2 Model #2: Patient Satisfaction (Classification)

In [ ]:
# Patient satisfaction model
X_sat = ml_data[feature_cols + ['Time_to_Doctor', 'Total_ER_Time']]
y_sat = ml_data['Patient Satisfaction']

X_train_sat, X_test_sat, y_train_sat, y_test_sat = train_test_split(
    X_sat, y_sat, test_size=0.2, random_state=42, stratify=y_sat)

scaler_sat = StandardScaler()
X_train_sat_sc = scaler_sat.fit_transform(X_train_sat)
X_test_sat_sc = scaler_sat.transform(X_test_sat)

rf_sat = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_sat.fit(X_train_sat_sc, y_train_sat)
y_pred_sat = rf_sat.predict(X_test_sat_sc)

accuracy_sat = (y_pred_sat == y_test_sat).mean()
print(f"✅ Satisfaction Model Accuracy: {accuracy_sat*100:.1f}%")
print("\nClassification Report:")
print(classification_report(y_test_sat, y_pred_sat))

### 7.3 Model #3: 15-Minute Threshold (Binary Classification)

In [ ]:
# 15-minute threshold model
X_15min = ml_data[feature_cols]
y_15min = (ml_data['Time_to_Doctor'] <= 15).astype(int)

X_train_15, X_test_15, y_train_15, y_test_15 = train_test_split(
    X_15min, y_15min, test_size=0.2, random_state=42, stratify=y_15min)

scaler_15 = StandardScaler()
X_train_15_sc = scaler_15.fit_transform(X_train_15)
X_test_15_sc = scaler_15.transform(X_test_15)

rf_15min = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_15min.fit(X_train_15_sc, y_train_15)
y_pred_15 = rf_15min.predict(X_test_15_sc)

accuracy_15 = (y_pred_15 == y_test_15).mean()
print(f"✅ 15-Min Threshold Model Accuracy: {accuracy_15*100:.1f}%")
print("\nClassification Report:")
print(classification_report(y_test_15, y_pred_15))

## 8. Scenario Analysis (What-If Simulations)Testing operational improvements to predict impact.

In [ ]:
# Calculate baseline metrics
current_avg_wait = visits['Time_to_Doctor'].mean()
current_satisfaction = visits['Patient Satisfaction'].mean()
current_15min_pct = visits['Seen_Within_15min'].mean() * 100

shift_performance = visits.groupby('Shift').agg({
    'Time_to_Doctor': 'mean',
    'Visit ID': 'count'
})

worst_shift = shift_performance['Time_to_Doctor'].idxmax()

print("CURRENT BASELINE:")
print(f"  Average wait time: {current_avg_wait:.1f} minutes")
print(f"  Average satisfaction: {current_satisfaction:.2f}/5.0")
print(f"  % within 15 min: {current_15min_pct:.1f}%")
print(f"  Worst shift: {worst_shift}")

In [ ]:
# Scenario 1: Add 1 doctor to worst shift
print("\nSCENARIO 1: ADD 1 DOCTOR TO WORST SHIFT")
worst_data = visits[visits['Shift'] == worst_shift].dropna(subset=['Patients_per_Doctor'])
current_ratio = worst_data['Patients_per_Doctor'].mean()
doctors_now = worst_data['Doctors On Duty'].mean()
new_ratio = current_ratio * doctors_now / (doctors_now + 1)
estimated_reduction_s1 = (current_ratio - new_ratio) * 2  # Conservative estimate
scenario1_wait = current_avg_wait - estimated_reduction_s1

print(f"  Current ratio: {current_ratio:.1f}:1 → New: {new_ratio:.1f}:1")
print(f"  Estimated reduction: {estimated_reduction_s1:.1f} minutes")
print(f"  New wait time: {scenario1_wait:.1f} minutes")

# Scenario 2: Improve triage efficiency by 20%
print("\nSCENARIO 2: IMPROVE TRIAGE EFFICIENCY BY 20%")
triage_time = visits['Triage_Duration'].mean()
triage_reduction = triage_time * 0.2
scenario2_wait = current_avg_wait - triage_reduction

print(f"  Time saved: {triage_reduction:.1f} minutes")
print(f"  New wait time: {scenario2_wait:.1f} minutes")

# Scenario 3: Combined approach
print("\nSCENARIO 3: COMBINED APPROACH")
combined_reduction = estimated_reduction_s1 + triage_reduction
combined_wait = current_avg_wait - combined_reduction
combined_improvement_pct = (combined_reduction / current_avg_wait) * 100

print(f"  Total reduction: {combined_reduction:.1f} minutes ({combined_improvement_pct:.1f}%)")
print(f"  New wait time: {combined_wait:.1f} minutes")

# Estimate satisfaction improvement
sat_corr = corr_matrix.loc['Time_to_Doctor', 'Patient Satisfaction']
sat_improvement = abs(sat_corr) * (combined_reduction / 10)
new_satisfaction = current_satisfaction + sat_improvement

print(f"  Estimated new satisfaction: {new_satisfaction:.2f}/5.0")

In [ ]:
# Visualize scenario comparisons
scenarios = {
    'Current': current_avg_wait,
    'Scenario 1\n+1 Doctor': scenario1_wait,
    'Scenario 2\nTriage': scenario2_wait,
    'Scenario 3\nCombined': combined_wait
}

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Wait time comparison
colors_scen = ['#e74c3c', '#3498db', '#f39c12', '#2ecc71']
bars = axes[0].bar(range(len(scenarios)), scenarios.values(), color=colors_scen, edgecolor='black', linewidth=2)
axes[0].set_xticks(range(len(scenarios)))
axes[0].set_xticklabels(scenarios.keys())
axes[0].set_ylabel('Average Wait Time (minutes)', fontsize=12, fontweight='bold')
axes[0].set_title('Wait Time Reduction Scenarios', fontsize=14, fontweight='bold')
axes[0].axhline(current_avg_wait, color='red', linestyle='--', linewidth=2, label='Current')
axes[0].legend()

for i, (bar, val) in enumerate(zip(bars, scenarios.values())):
    reduction = current_avg_wait - val
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f'{val:.0f}\n(-{reduction:.0f})', ha='center', va='bottom', fontweight='bold')

# Satisfaction improvement
sat_scenarios = {'Current': current_satisfaction, 'Scenario 3': new_satisfaction}
bars2 = axes[1].bar(range(len(sat_scenarios)), sat_scenarios.values(),
                    color=['#e74c3c', '#2ecc71'], edgecolor='black', linewidth=2)
axes[1].set_xticks(range(len(sat_scenarios)))
axes[1].set_xticklabels(sat_scenarios.keys())
axes[1].set_ylabel('Patient Satisfaction', fontsize=12, fontweight='bold')
axes[1].set_title('Satisfaction Improvement', fontsize=14, fontweight='bold')
axes[1].set_ylim(0, 5)
axes[1].axhline(4.0, color='green', linestyle='--', linewidth=2, label='Target: 4.0')
axes[1].legend()

for bar, val in zip(bars2, sat_scenarios.values()):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                 f'{val:.2f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('outputs/scenario_analysis.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Saved: outputs/scenario_analysis.png")

## 9. Export Results for Presentation

In [ ]:
# Export summary statistics
summary_stats = pd.DataFrame({
    'Metric': [
        'Total Visits Analyzed',
        'Average Daily Volume',
        'Average Time to Doctor (min)',
        '% Seen Within 15 Min',
        'Average Total ER Time (hours)',
        'Average Patient Satisfaction',
        'Worst Performing Shift',
        'ML Model Accuracy (Wait Time)',
        'ML Model Accuracy (Satisfaction)'
    ],
    'Value': [
        f"{len(visits):,}",
        f"{len(visits) / visits['Date_dt'].nunique():.0f}",
        f"{current_avg_wait:.1f}",
        f"{current_15min_pct:.1f}%",
        f"{visits['Total_ER_Time'].mean()/60:.2f}",
        f"{current_satisfaction:.2f}/5.0",
        worst_shift,
        f"±{results[best_model_name]['MAE']:.1f} min",
        f"{accuracy_sat*100:.1f}%"
    ]
})

summary_stats.to_csv('outputs/summary_statistics.csv', index=False)
print("✅ Saved: outputs/summary_statistics.csv")

# Export shift analysis
shift_analysis = visits.groupby('Shift').agg({
    'Visit ID': 'count',
    'Time_to_Doctor': 'mean',
    'Total_ER_Time': 'mean',
    'Seen_Within_15min': 'mean',
    'Patient Satisfaction': 'mean'
}).round(2)

shift_analysis.to_csv('outputs/shift_analysis.csv')
print("✅ Saved: outputs/shift_analysis.csv")

print("\n📊 All visualizations saved in outputs/ folder:")
print("  • wait_time_analysis.png")
print("  • correlation_matrix.png")
print("  • scenario_analysis.png")
print("  • summary_statistics.csv")
print("  • shift_analysis.csv")

## 10. Executive Summary### Key Findings & Recommendations

In [ ]:
print("="*80)
print("EXECUTIVE SUMMARY")
print("="*80)

print("\n1. CURRENT PERFORMANCE GAPS")
print(f"  ❌ Only {current_15min_pct:.1f}% seen within 15 minutes (Target: 40%+)")
print(f"  ❌ Average wait: {current_avg_wait:.1f} minutes (Target: <15 minutes)")
print(f"  ⚠️  Satisfaction: {current_satisfaction:.2f}/5.0 (Target: 4.0+)")

print("\n2. PRIMARY BOTTLENECKS")
bottlenecks = [
    ('Wait to Doctor', visits['Wait_to_Doctor'].mean()),
    ('Treatment Duration', visits['Treatment_Duration'].mean()),
    ('Triage Duration', visits['Triage_Duration'].mean())
]
for i, (stage, time) in enumerate(bottlenecks, 1):
    print(f"  {i}. {stage}: {time:.1f} minutes")

print("\n3. STAFFING INSIGHTS")
print(f"  ⚠️  {worst_shift} shift has longest waits")
print(f"  📊 Correlation: More patients per doctor = longer waits")

print("\n4. MACHINE LEARNING INSIGHTS")
print(f"  ✅ Can predict wait times within ±{results[best_model_name]['MAE']:.1f} minutes")
print(f"  ✅ Can predict satisfaction with {accuracy_sat*100:.1f}% accuracy")
print(f"  ✅ Can predict 15-min threshold with {accuracy_15*100:.1f}% accuracy")

print("\n5. RECOMMENDATIONS")
print("  HIGH PRIORITY:")
print(f"    • Increase staffing during {worst_shift} shift")
print("    • Reduce 'Wait to Doctor' - biggest bottleneck")
print("    • Optimize fast-track triage")
print("  MEDIUM PRIORITY:")
print("    • Standardize triage protocols")
print("    • Extend fast-track hours beyond 11 PM")
print("    • Use ML models for predictive scheduling")

print("\n6. PROJECTED IMPACT")
print(f"  📈 Reduce wait time by {combined_improvement_pct:.0f}% (to ~{combined_wait:.0f} min)")
print(f"  📈 Increase 15-min compliance from {current_15min_pct:.0f}% to 60-70%")
print(f"  📈 Improve satisfaction from {current_satisfaction:.2f} to {new_satisfaction:.2f}")

print("\n" + "="*80)
print("Analysis Complete!")
print("="*80)